In [24]:
using LinearAlgebra
using SpecialFunctions
using Random
using ForwardDiff
using MultiQuad

include("Misc.jl");
include("GreensFunction.jl");
include("InteractionMatrix.jl");
include("Fields.jl");
include("ComplexGradient.jl")
include("Power.jl")

powerout (generic function with 1 method)

In [25]:
ω = 1.
J = Stdd(2);

scattpos = [[1. 2. 3.]
            [2. 3. 4.]
            [3. 3. 3.]
            [2. 1. 3.]];

alphas = [(1. + 2im)
          (1. - 2im)
          (2. + 1im)
          (1. - 3im)];

ϕinput = [(1. +1im), 2., 3., 2.];

x = [1., 1., 0.];

In [26]:
f(ϕ, θ) = integrand([1., θ, ϕ], ϕinput, scattpos, alphas, ω, J)[1]
f(2, 6)

-0.008161290384855783

In [27]:
powerout(3, ϕinput, scattpos, alphas, ω, J::Stdd; normalized=false, imagshift=1E-12)

(-5.546590904080472, 8.253120983210979e-8)

In [28]:
powerout(2, ϕinput, scattpos, alphas, ω, J::Stdd; normalized=false, imagshift=1E-12)

(-1.6630709713999716, 2.475236003646746e-8)